In [1]:
import pandas as pd
import numpy as np


#### Data Source
https://www.himalayandatabase.com/index.html

##### Guide
https://www.himalayandatabase.com/downloads/HD%20Online%20Guide.pdf

In [2]:
df_peaks = pd.read_csv("./Himalayan Database/RawData/peaksall.csv")
df_ex = pd.read_csv("./Himalayan Database/RawData/expeditions.csv")
df_mem = pd.read_csv("./Himalayan Database/RawData/members.csv")

C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_mem.columns

Index(['expid', 'membid', 'peakid', 'myear', 'mseason', 'fname', 'lname',
       'sex', 'age', 'birthdate', 'yob', 'calcage', 'citizen', 'status',
       'residence', 'occupation', 'leader', 'deputy', 'bconly', 'nottobc',
       'support', 'disabled', 'hired', 'sherpa', 'tibetan', 'msuccess',
       'mclaimed', 'mdisputed', 'msolo', 'mtraverse', 'mski', 'mparapente',
       'mspeed', 'mhighpt', 'mperhighpt', 'msmtdate1', 'msmtdate2',
       'msmtdate3', 'msmttime1', 'msmttime2', 'msmttime3', 'mroute1',
       'mroute2', 'mroute3', 'mascent1', 'mascent2', 'mascent3', 'mo2used',
       'mo2none', 'mo2climb', 'mo2descent', 'mo2sleep', 'mo2medical',
       'mo2note', 'death', 'deathdate', 'deathtime', 'deathtype', 'deathhgtm',
       'deathclass', 'msmtbid', 'msmtterm', 'hcn', 'mchksum', 'host', 'comrte',
       'stdrte', 'route1', 'route2', 'route3', 'route4', 'nation', 'leaders',
       'sponsor', 'termreason', 'totmembers', 'smtmembers', 'mdeaths',
       'tothired', 'nohired', 'smthire

### Countries

In [4]:
print(list(set(("/").join(df_mem.citizen.dropna().unique()).split("/"))))

['Argentina', 'Ukraine', 'Uzbekistan', 'S Korea', 'Norway', 'Zambia', 'Germany', 'Hong Kong', 'Ireland', 'Cyprus', 'Russia', 'Dominican Republic', 'W Germany', 'Turkey', 'Netherlands', 'Israel', 'Kenya', 'Egypt', 'New Zealand', 'Myanmar', 'Australia', 'UK', 'Slovenia', 'Mexico', 'USSR', 'Czechoslovakia', 'Yugoslavia', 'India?', 'Moldova', 'Jordan', 'S Africa', 'Azerbaijan', 'Portugal', 'Qatar', 'Taiwan', 'Tunisia', 'Finland', 'Switzerland', 'Sweden', 'Mongolia', 'Malaysia', 'Armenia', 'Mauritius', 'Iran', 'Georgia', 'Vietnam', 'France', 'Serbia', 'Denmark', 'Montenegro', 'Liechtenstein', 'Slovakia', 'Nepal', 'Luxembourg', 'Spain', 'Palestine', 'Kuwait', 'Thailand', 'Brazil', 'Bolivia', 'Panama', 'USA', 'Malta', 'N Korea', 'Japan', 'Czech Republic', 'Greece', 'San Marino', 'Bangladesh', 'Hungary', 'Ecuador', 'Costa Rica', 'Jamaica', 'Iceland', 'Croatia', 'Belgium', 'Turkmenistan', 'Estonia', 'Colombia', 'Guatemala', 'Lebanon', 'Bosnia-Herzegovina', 'Peru', 'Kyrgyz Republic', 'Poland', '

In [5]:
##df_mem.citizen.apply(lambda x : coco.convert(names=x, to='ISO2') if manual_map.get(x) == None  else manual_map.get(x))

### Get Members ready to join

In [6]:
df_mem['female']  = df_mem.sex.apply(lambda x : True if x=="F" else False)

In [7]:
df_mem.mspeed.value_counts()

False    65521
True        13
Name: mspeed, dtype: int64

In [8]:
gdf_mem = df_mem.groupby('peakid').agg({'expid' : 'count' ,'death' : 'sum', 'female' : 'sum', 'calcage' : 'mean', 'citizen' : 'unique','mo2used' : 'sum', "route1" : pd.Series.mode})

#### Grouby only the succeeding members

In [9]:
gdf_mem_success = df_mem[df_mem.msuccess == True].groupby('peakid').agg({'citizen' : 'unique', 'female' : 'sum','expid' : 'count' })

In [10]:
gdf_mem_success 

,citizen,female,expid
peakid,,,
ACHN,"[France, Nepal, Japan]",1.0,11
AMAD,"[W Germany, USA, Nepal, France, New Zealand, J...",322.0,3540
AMPG,"[UK, Nepal]",1.0,10
AMPH,[S Korea],1.0,3
AMPM,[Germany],1.0,5
...,...,...,...
URKM,"[Japan, Nepal, USA, Austria, France]",3.0,23
URMA,"[Taiwan, S Korea, Singapore, Australia, Nepal,...",11.0,52
YAKA,"[Japan, Nepal]",0.0,4


In [11]:
#gdf_mem.columns = ['_'.join(col).strip() for col in gdf_mem.columns.values]
#gdf_mem_success.columns = ['_'.join(col).strip() for col in gdf_mem_success.columns.values]

In [12]:
## Rename columns accordingly

In [13]:
gdf_mem.rename(columns={'expid' : 'count_mem_try', "female" :  "count_mem_try_female", "calcage" :"mean_mem_try_age", "citizen" : "country_mem_try", "route1" : "most_pop_route" }, inplace=True)
gdf_mem_success.rename(columns={'expid' : 'count_mem_success', "female" :  "count_mem_success_female", "calcage" :"mean_mem_success_age", "citizen" : "country_mem_success" }, inplace=True)

In [14]:
gdf_mem = pd.merge(gdf_mem,gdf_mem_success, on="peakid", how="left") 

In [15]:
gdf_mem.reset_index(inplace=True)

### Merge with Peak data

In [16]:
df = pd.merge(df_peaks,gdf_mem, on='peakid', how="left")

In [17]:
## WOrk with the ones with coords
df.dropna(subset=["lng"], inplace=True)

##### Adding ISO the the countries

In [18]:
import country_converter as coco

manual_map = {"USSR": 'USSR', "UK" : "GB", "UAE" : "AE", "W Germany" : "DE", "S Africa" : "ZA", "Yugoslavia" : "YUG" }


In [19]:
df.country_mem_success = df.country_mem_success.apply(lambda x : x if isinstance(x,float) else sum([i.split("/") for i in x ], []))

In [20]:
df.country_mem_success = df.country_mem_success.apply(lambda x : [ c if manual_map.get(c) == None else manual_map.get(c) for c in x ] if not isinstance(x,float) else x)

In [21]:
df.country_mem_success = df.country_mem_success.apply(lambda x : coco.convert(names=x, to='ISO2' , not_found=None) if not isinstance(x,float) else x)

YUG not found in ISO3
USSR not found in regex
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3


In [22]:
df.country_mem_success[~df.country_mem_success.isna()].apply(len)

0      151
1       45
2       54
3       11
4       46
      ... 
456      2
458      2
462      3
463      2
465      3
Name: country_mem_success, Length: 226, dtype: int64

### Line Chart data create

In [23]:
df_temp = df_mem.groupby(['peakid', 'myear']).agg({'membid' : 'count','msuccess' : 'sum','death' : 'sum'})

In [24]:
df_temp = df_temp.reset_index().sort_values(["peakid", 'myear'], ascending=[True,True]).groupby(['peakid' ]).agg(list).rename(columns={"myear" : "lineX", "membid" : "lineY_attemps", "msuccess" : "lineY_success", "death" : "lineY_death"} )

In [25]:
df = pd.merge(df,df_temp, on='peakid', how="left")

In [26]:
df

,peakid,pkname,pkname2,location,heightm,heightf,himal,region,open,unlisted,...,country_mem_try,mo2used,most_pop_route,country_mem_success,count_mem_success_female,count_mem_success,lineX,lineY_attemps,lineY_success,lineY_death
0,EVER,Everest,"Sagarmatha, Chomolungma, Qomolungma",Khumbu Himal,8850,29035,12,2,True,False,...,"[USA, USA/Switzerland, UK, India, Nepal, Japan...",9144.0,S Col-SE Ridge,"[US, IN, NP, IT, JP, GB, KR, CH, DE, PL, FR, A...",417.0,6927.0,"[1921, 1922, 1924, 1933, 1934, 1935, 1936, 193...","[13, 22, 13, 43, 4, 18, 33, 23, 3, 5, 14, 46, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 7.0, 4.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,KANG,Kangchenjunga,NaN,Kangchenjunga Himal,8586,28169,9,1,True,False,...,"[USA, India, France, UK, Japan, Nepal, W Germa...",246.0,SW Face,"[IN, GB, JP, NP, DE, CZ, FR, IT, AT, CH, CA, P...",12.0,332.0,"[1905, 1910, 1920, 1925, 1929, 1930, 1931, 193...","[9, 2, 2, 1, 15, 15, 16, 3, 1, 2, 2, 8, 17, 23...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.0, 0.0, 0.0, 0.0, 1.0, 1.0, 4.0, 0.0, 0.0, ..."
2,LHOT,Lhotse,NaN,Khumbu Himal,8516,27940,12,2,True,False,...,"[USA, USA/Switzerland, Austria, Switzerland, W...",551.0,W Face,"[CH, DE, AT, NP, PL, BG, JP, IT, CZ, YUG, KR, ...",44.0,603.0,"[1955, 1956, 1972, 1973, 1974, 1975, 1976, 197...","[11, 12, 3, 9, 22, 16, 21, 14, 1, 36, 33, 47, ...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
3,YALU,Yalung Kang,Yalungkar,Kangchenjunga Himal (W summit of Kangchenjunga),8505,27904,9,1,True,False,...,"[Japan, Austria, W Germany, Spain, Mexico, Aus...",53.0,SE Face,"[JP, DE, AT, MX, PL, FR, YUG, ES, KR, NP, USSR]",1.0,53.0,"[1973, 1974, 1975, 1978, 1980, 1981, 1984, 198...","[16, 13, 12, 14, 13, 24, 17, 16, 7, 17, 34, 6,...","[2.0, 0.0, 9.0, 0.0, 1.0, 5.0, 8.0, 2.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, ..."
4,MAKA,Makalu,NaN,Makalu Himal,8485,27838,14,2,True,False,...,"[Japan, Yugoslavia, Czechoslovakia, Austria, W...",357.0,Makalu La-NW Ridge,"[JP, YUG, CZ, ES, DE, AT, CH, NP, US, PL, KR, ...",27.0,421.0,"[1954, 1955, 1961, 1969, 1970, 1971, 1972, 197...","[31, 15, 18, 5, 20, 14, 14, 22, 20, 21, 32, 21...","[0.0, 9.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,KHON,Khongma Tse,Mehra Peak,Khumbu Himal (SW of Nuptse),5798,19022,12,2,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,POKA,Pokalde,NaN,Khumbu Himal (N of Ama Dablam),5780,18963,12,2,False,False,...,"[UK, Japan, Iran]",0.0,Kongma La-N Ridge,"[GB, JP]",0.0,12.0,"[1953, 1954, 1970, 1973, 1976]","[4, 6, 1, 1, 4]","[3.0, 5.0, 1.0, 0.0, 3.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
366,THAR,Tharpu Chuli,Tent Peak,Annapurna Himal (Annapurna Sanctuary),5695,18684,1,5,False,False,...,"[UK, Japan, W Germany, Nepal]",0.0,"[From S Annapurna Glacier, NW Ridge]","[JP, DE, NP]",0.0,5.0,"[1956, 1964, 1965]","[1, 3, 3]","[0.0, 2.0, 3.0]","[0.0, 0.0, 0.0]"
367,AMPG,Amphu Gyabjen,NaN,Khumbu Himal (N of Ama Dablam),5630,18471,12,2,True,False,...,"[UK, Nepal]",4.0,S Ridge,"[GB, NP]",1.0,10.0,"[1953, 1959, 2010]","[4, 6, 3]","[4.0, 6.0, 0.0]","[0.0, 0.0, 0.0]"


In [27]:
df.to_json("./Himalayan Database/GeoData/peaks2.json", orient='records')

In [28]:
df.to_csv("./Himalayan Database/GeoData/peaks2.csv")

In [32]:
df['value'] = df.apply(lambda x : [x.lat,x.lng, x.peakid], axis=1)
df['text'] = df.pkname + " - " + df.heightm.astype("str") + "m"

In [34]:
df[['value', 'text']].to_json("./assets/data/peaks_search.json" ,orient="records")

In [31]:
geojson = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [d["lon"], d["lat"]],
            },
        "properties" : d,
     } for d in data]
}

NameError: name 'data' is not defined

In [ ]:
df.to_dict(orient="records")